# Entrenamiento de Detector de Logos de Marcas - BALANCEADO

Este notebook:
1. Monta Google Drive
2. Descomprime dataset
3. **BALANCEA el dataset** usando oversampling con augmentation
4. Entrena YOLOv8
5. Guarda el mejor modelo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Descomprimir dataset original
import os

zip_path = "/content/drive/MyDrive/datasets/merged_dataset.zip"
dest_dir = "/content/merged_dataset"

os.makedirs(dest_dir, exist_ok=True)
!unzip -q "{zip_path}" -d "{dest_dir}"

print("Dataset descomprimido")

Dataset descomprimido


In [3]:
# Instalar dependencias
!pip install -q ultralytics opencv-python-headless

print("Dependencias instaladas")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.1 MB/s eta 0:00:00
Dependencias instaladas


## Script de Balanceo de Dataset

Este script aplicará oversampling a las clases minoritarias usando data augmentation.

In [4]:
import os
import shutil
import random
import cv2
import numpy as np
from collections import Counter

# Configuracion
BASE_DIR = "/content/merged_dataset"
OUTPUT_DIR = "/content/merged_dataset_balanced"
MIN_IMAGES_PER_CLASS = 800

def read_yolo_label(label_path):
    """Lee archivo de labels YOLO"""
    if not os.path.exists(label_path):
        return []
    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id = int(parts[0])
                coords = [float(x) for x in parts[1:]]
                bboxes.append([class_id] + coords)
    return bboxes

def write_yolo_label(label_path, bboxes):
    """Escribe bboxes en formato YOLO"""
    with open(label_path, 'w') as f:
        for bbox in bboxes:
            class_id = int(bbox[0])
            coords = ' '.join([f"{x:.6f}" for x in bbox[1:]])
            f.write(f"{class_id} {coords}\n")

def apply_random_augmentation(image, bboxes):
    """Aplica augmentation aleatoria"""
    technique = random.choice(['flip', 'brightness', 'rotation', 'blur', 'noise'])

    if technique == 'flip':
        aug_image = cv2.flip(image, 1)
        aug_bboxes = []
        for bbox in bboxes:
            new_bbox = bbox.copy()
            new_bbox[1] = 1.0 - bbox[1]
            aug_bboxes.append(new_bbox)
        return aug_image, aug_bboxes

    elif technique == 'brightness':
        factor = random.uniform(0.7, 1.3)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
        hsv[:, :, 2] = np.clip(hsv[:, :, 2] * factor, 0, 255)
        aug_image = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)
        return aug_image, bboxes

    elif technique == 'rotation':
        angle = random.uniform(-15, 15)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
        aug_image = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REFLECT)
        return aug_image, bboxes

    elif technique == 'blur':
        kernel_size = random.choice([3, 5])
        aug_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
        return aug_image, bboxes

    elif technique == 'noise':
        noise_level = random.uniform(5, 15)
        noise = np.random.normal(0, noise_level, image.shape).astype(np.uint8)
        aug_image = cv2.add(image, noise)
        return aug_image, bboxes

    return image, bboxes

def count_classes(base_dir, split):
    """Cuenta imagenes por clase"""
    labels_dir = os.path.join(base_dir, split, "labels")
    if not os.path.exists(labels_dir):
        return Counter()

    class_counts = Counter()
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith('.txt'):
            continue
        label_path = os.path.join(labels_dir, label_file)
        bboxes = read_yolo_label(label_path)
        for bbox in bboxes:
            class_counts[int(bbox[0])] += 1
    return class_counts

print("Funciones definidas")

Funciones definidas


In [5]:
# Ejecutar balanceo
print("="*60)
print("BALANCEO DE DATASET")
print("="*60)

os.makedirs(OUTPUT_DIR, exist_ok=True)

for split in ['train', 'valid', 'test']:
    print(f"\nProcesando {split}...")

    src_images_dir = os.path.join(BASE_DIR, split, "images")
    src_labels_dir = os.path.join(BASE_DIR, split, "labels")
    dst_images_dir = os.path.join(OUTPUT_DIR, split, "images")
    dst_labels_dir = os.path.join(OUTPUT_DIR, split, "labels")

    os.makedirs(dst_images_dir, exist_ok=True)
    os.makedirs(dst_labels_dir, exist_ok=True)

    # Contar clases originales
    class_counts = count_classes(BASE_DIR, split)
    print(f"Distribucion original: {dict(class_counts)}")

    # Mapear imagenes por clase
    class_to_images = {i: [] for i in range(14)}

    for label_file in os.listdir(src_labels_dir):
        if not label_file.endswith('.txt'):
            continue

        image_file = label_file.replace('.txt', '.jpg')
        image_path = os.path.join(src_images_dir, image_file)
        label_path = os.path.join(src_labels_dir, label_file)

        if not os.path.exists(image_path):
            continue

        bboxes = read_yolo_label(label_path)
        for bbox in bboxes:
            class_id = int(bbox[0])
            class_to_images[class_id].append({
                'image': image_path,
                'label': label_path,
                'bboxes': bboxes
            })

    # Copiar originales
    for label_file in os.listdir(src_labels_dir):
        image_file = label_file.replace('.txt', '.jpg')
        src_img = os.path.join(src_images_dir, image_file)
        src_lbl = os.path.join(src_labels_dir, label_file)

        if os.path.exists(src_img):
            shutil.copy2(src_img, os.path.join(dst_images_dir, image_file))
            shutil.copy2(src_lbl, os.path.join(dst_labels_dir, label_file))

    # Aplicar oversampling
    target_count = MIN_IMAGES_PER_CLASS

    for class_id, images in class_to_images.items():
        current_count = len(images)
        if current_count == 0 or current_count >= target_count:
            continue

        needed = target_count - current_count
        print(f"  Clase {class_id}: {current_count} -> generando {needed}")

        for i in range(needed):
            sample = random.choice(images)
            image = cv2.imread(sample['image'])
            if image is None:
                continue

            bboxes = sample['bboxes']
            aug_image, aug_bboxes = apply_random_augmentation(image, bboxes)

            base_name = os.path.basename(sample['image']).replace('.jpg', '')
            new_name = f"{base_name}_aug_{i}"

            cv2.imwrite(os.path.join(dst_images_dir, f"{new_name}.jpg"), aug_image)
            write_yolo_label(os.path.join(dst_labels_dir, f"{new_name}.txt"), aug_bboxes)

    # Contar resultado
    final_counts = count_classes(OUTPUT_DIR, split)
    print(f"Distribucion final: {dict(final_counts)}")

print("\nBalanceo completado!")

BALANCEO DE DATASET

Procesando train...
Distribucion original: {1: 1776, 2: 1501, 7: 566, 13: 1908, 8: 1476, 12: 2922, 10: 1091, 0: 1144, 5: 1914, 3: 563, 4: 370, 6: 362, 9: 121, 11: 139}
  Clase 3: 563 -> generando 237
  Clase 4: 370 -> generando 430
  Clase 6: 362 -> generando 438
  Clase 7: 566 -> generando 234
  Clase 9: 121 -> generando 679
  Clase 11: 139 -> generando 661
Distribucion final: {11: 865, 1: 1776, 2: 1501, 7: 808, 3: 871, 13: 1908, 8: 1476, 12: 2922, 10: 1091, 9: 831, 0: 1144, 5: 1914, 4: 1078, 6: 925}

Procesando valid...
Distribucion original: {1: 140, 5: 250, 11: 22, 12: 347, 7: 81, 0: 103, 6: 62, 10: 93, 13: 330, 8: 172, 2: 44, 9: 38, 3: 81, 4: 76}
  Clase 0: 103 -> generando 697
  Clase 1: 140 -> generando 660
  Clase 2: 44 -> generando 756
  Clase 3: 81 -> generando 719
  Clase 4: 76 -> generando 724
  Clase 5: 250 -> generando 550
  Clase 6: 62 -> generando 738
  Clase 7: 81 -> generando 719
  Clase 8: 172 -> generando 628
  Clase 9: 38 -> generando 762
  Cla

In [6]:
# Crear data.yaml corregido
data_yaml_content = f"""path: {OUTPUT_DIR}

train: train/images
val: valid/images
test: test/images

names:
  0: Audi
  1: BMW
  2: Chevrolet
  3: Ford
  4: Honda
  5: Hyundai
  6: KIA
  7: Mazda
  8: Mercedes
  9: Mitsubishi
  10: Nissan
  11: Suzuki
  12: Toyota
  13: Volkswagen
"""

data_yaml_path = os.path.join(OUTPUT_DIR, "data.yaml")
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"data.yaml guardado en: {data_yaml_path}")
print("\n✓ Dataset balanceado listo para entrenamiento")

data.yaml guardado en: /content/merged_dataset_balanced/data.yaml

✓ Dataset balanceado listo para entrenamiento


## Entrenamiento con Dataset Balanceado

In [7]:
from ultralytics import YOLO

# Cargar modelo
model = YOLO("yolov8s.pt")

# Entrenar con dataset balanceado y augmentation
results = model.train(
    data=data_yaml_path,
    imgsz=640,
    epochs=80,
    batch=16,
    patience=15,
    save_period=10,
    # Augmentation adicional
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.3,
    # Hiperparametros
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    # Proyecto
    project="car_brand_balanced",
    name="yolov8s_balanced",
    workers=2
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged_dataset_balanced/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, i

In [8]:
# Guardar mejor modelo en Drive
best_model_path = "/content/car_brand_balanced/yolov8s_balanced/weights/best.pt"
save_path = "/content/drive/MyDrive/datasets/models/brand_detector_balanced.pt"

os.makedirs(os.path.dirname(save_path), exist_ok=True)
!cp "{best_model_path}" "{save_path}"

print(f"✓ Modelo guardado en: {save_path}")

✓ Modelo guardado en: /content/drive/MyDrive/datasets/models/brand_detector_balanced.pt
